In [19]:
import keras
# from keras.applications import inception_v3

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K


In [42]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(300,300,3), pooling = 'avg')

In [43]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1035 (Conv2D)            (None, 149, 149, 32) 864         input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1035 (Batch (None, 149, 149, 32) 96          conv2d_1035[0][0]                
__________________________________________________________________________________________________
activation_1035 (Activation)    (None, 149, 149, 32) 0           batch_normalization_1035[0][0]   
__________________________________________________________________________________________________
conv2d_103

In [45]:
base_model.output

<tf.Tensor 'global_average_pooling2d_4/Mean:0' shape=(?, 2048) dtype=float32>

In [52]:
# add a global spatial average pooling layer
x = base_model.output
# let's add a fully-connected layer
preds = Dense(300, activation='relu')(x)

In [49]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [51]:
# build model
meme_model = Model(inputs=base_model.input, outputs=preds)
meme_model.summary()
# compile the model (should be done *after* setting layers to non-trainable)
meme_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [3]:
# train the model on the new data for a few epochs
model.fit_generator(...)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)

NameError: name 'inception_v3' is not defined